# Embedding dimension

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Pool
import time
from tqdm import tqdm

from graph_generation import create_ER
from models import GCN, GCNEmbed
from train import Trainer
from embedding import Embedding

import torch
from torch_geometric.loader import DataLoader
device = torch.device('cuda')

import yaml

In [11]:
config = yaml.safe_load(open("configs.yml"))

In [12]:
config.items()

dict_items([('graph_dataset', {'ERmodel': True, 'p1ER': 0.3, 'p2ER': 0.08, 'Num_nodes': 100, 'Num_grafi_per_tipo': 100}), ('memcached', {'host': '10.0.0.99'}), ('workers', [{'host': '10.0.0.101', 'port': 2301}, {'host': '10.0.0.102', 'port': 2302}])])

In [ ]:
criterion = nn.MSELoss()
p1 = config['graph_dataset']['p1ER']
p2 = config['graph_dataset']['p2ER']
Num_grafi_per_tipo = config['graph_dataset']['Num_grafi_per_tipo']

grafi_0 = create_ER(N, p1, Num_grafi_per_tipo)
grafi_1 = create_ER(N, p2, Num_grafi_per_tipo)
dataset_grafi_nx = grafi_0 + grafi_1
dataset_labels = np.array([0]*len(grafi_0) + [1]*len(grafi_1))


def experiment_embedding(config):

    
    model = GCN(hidden_channels=num_neurons, num_neurons_last_layer=num_last_neurons)
    model.to(device)
    #print(model)
    
    trainer = Trainer(model, 
                      learning_rate=learning_rate, 
                      epochs=epochs, 
                      batch_size=batch_size, 
                      layers = layers, 
                      neurons=num_neurons, 
                      last_layer_neurons=num_last_neurons)
    print("Loading Dataset...")
    trainer.load_dataset(dataset_grafi_nx, dataset_labels, percentage_train=percentage_train)
    train_loss_list, test_loss_list = trainer.launch_training()
    
    # get embedding of all graphs in dataset
    embed_model = GCNEmbed(model)
    embed_model = embed_model.to(device)
    whole_data = trainer.dataset.dataset_pyg
    all_data_loader = DataLoader(whole_data, batch_size=trainer.dataset.len_data, shuffle=False)
    batch = next(iter(all_data_loader))
    embeddings_array = embed_model(batch.x, batch.edge_index, batch.batch).cpu().detach().numpy()
    
    embeddings = Embedding(embeddings_array, trainer.dataset.labels, (p1,p2), test_loss_list)
    embeddings.calc_coppie()
    embeddings.calc_distances()
    
    return embeddings